In [ ]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nengo
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer
from scipy.stats import zscore

import phd

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

def img(array):
    array = zscore(array, axis=0)
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)

def img_compare(array1, array2):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    img(array1)
    plt.subplot(1, 2, 2)
    img(array2)
    plt.tight_layout()

timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

In [ ]:
timit = phd.timit.TIMIT(timit_path)
samples = timit.word_samples(['dirty'])

In [ ]:
model = phd.sermo.AudioFeatures()
model.fs = timit.fs
model.audio = samples['dirty'][0]
model.freqs = phd.filters.erbspace(20, 4000, 64)
model.n_cepstra = 13
print(model.t_audio)

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
mfcc = model.mfcc()
print mfcc.shape

In [ ]:
# Get NCCs
model.periphery.auditory_filter = phd.filters.gammatone(model.freqs)
net = model.build()

with net:
    c_p = nengo.Probe(net.cepstra.output, synapse=0.01)

sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)
ncc = sim.data[c_p]
print ncc.shape

In [ ]:
img_compare(mfcc, ncc)

## Linear interpolation / neighbourhood mean

In [ ]:
import scipy
from scipy.interpolate import interp1d

def lengthen(feature, n_frames):
    interp_x = np.linspace(0, n_frames, feature.shape[0])
    f = interp1d(interp_x, feature, axis=0, assume_sorted=True)
    return f(np.arange(n_frames))

def shorten(feature, n_frames):
    scale = int(feature.shape[0] / n_frames)
    pad_size = int(np.ceil(float(feature.shape[0]) / scale) * scale - feature.shape[0])
    feature = np.vstack([feature, np.zeros((pad_size, feature.shape[1])) * np.nan])
    out = scipy.nanmean(feature.reshape(-1, scale, feature.shape[1]), axis=1)
    return out[:n_frames]

In [ ]:
img_compare(mfcc, lengthen(mfcc, ncc.shape[0]))

In [ ]:
img_compare(shorten(ncc, mfcc.shape[0]), ncc)

## `scipy.ndimage.interpolate.zoom`

In [ ]:
from scipy.ndimage.interpolation import zoom

def lengthen(feature, n_frames):
    return zoom(feature, [float(n_frames) / feature.shape[0], 1])

def shorten(feature, n_frames):
    return zoom(feature, [float(n_frames) / feature.shape[0], 1])

In [ ]:
img_compare(mfcc, lengthen(mfcc, ncc.shape[0]))

In [ ]:
img_compare(shorten(ncc, mfcc.shape[0]), ncc)

## `scipy.signal.resample`

In [ ]:
from scipy.signal import resample

def lengthen(feature, n_frames):
    return resample(feature, n_frames)

def shorten(feature, n_frames):
    return resample(feature, n_frames)

In [ ]:
img_compare(mfcc, lengthen(mfcc, ncc.shape[0]))

In [ ]:
img_compare(shorten(ncc, mfcc.shape[0]), ncc)